# Deep Learning for tensiometry of pendant drop

### Import necessary packages and self-defined classes

In [3]:
# pyplot
import matplotlib.pyplot as plt
# pytorch
import torch
import torch.nn.functional as F  # useful stateless functions
# os
import os
# self-defined model
from model_pic import *
from model_cal import *
import pandas as pd

### Dataset loading and preprocessing

In [77]:
# To be done
data_matrix = df = pd.DataFrame()

for Data_point in range(1,228):

    d=pd.read_csv("Data_folder/" + str(Data_point) + ".csv", header= None)

    st = d[0]
    volume0 = d[1]
    rneedle = d[2]
    rr_zz = d.loc[0][3:]

    data_matrix = data_matrix.append(d, ignore_index= True)

    
print(data_matrix)
train_loader = []
val_loader = []
test_loader = []

     0    1    2             3         4         5         6         7    \
0    2.3  0.2    1  1.507010e-26  0.012809  0.025618  0.038426  0.051234   
1    2.4  0.2    1 -1.908640e-26  0.012808  0.025616  0.038423  0.051230   
2    2.5  0.2    1  1.076793e-27  0.012807  0.025614  0.038421  0.051226   
3    2.6  0.2    1 -2.257969e-27  0.012806  0.025613  0.038418  0.051223   
4    2.7  0.2    1 -2.748220e-26  0.012806  0.025611  0.038416  0.051220   
..   ...  ...  ...           ...       ...       ...       ...       ...   
222  6.1  0.8    1  3.218298e-26  0.014556  0.029109  0.043657  0.058197   
223  6.1  0.9    1 -6.731090e-26  0.014972  0.029942  0.044904  0.059857   
224  6.1  1.0    1 -1.194378e-25  0.015408  0.030812  0.046209  0.061594   
225  6.1  1.1    1  8.917468e-26  0.015859  0.031713  0.047558  0.063389   
226  6.1  1.2    1  1.298461e-25  0.016321  0.032637  0.048941  0.065229   

          8         9    ...       155       156       157       158  \
0    0.064040  

### Deside to apply host or device

In [18]:
USE_GPU = True
if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

print(device)

cuda


### Define the functions we need

In [4]:
def loss_cal_per_epoch(loss_list, model, loader) -> list:
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        loss_epoch = 0
        for t, (x, y) in enumerate(loader):
            x = x.to(device=device, dtype=torch.float32)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.float32)
            scores = model(x)
            loss = F.mse_loss(scores, y)
            loss_epoch += loss
        loss_list.append(loss_epoch/t)

def train(model, optimizer, epochs=1):
    loss_train = []
    loss_val = []
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(train_loader):
            model.train()  # put model to training mode
            x_noisy = x_noisy.to(device=device, dtype=torch.float32)  # move to device, e.g. GPU
            x_clean = x_clean.to(device=device, dtype=torch.float32)

            scores = model(x)
            loss = F.mse_loss(scores, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        loss_cal_per_epoch(loss_train, model, train_loader)
        loss_cal_per_epoch(loss_val, model, val_loader)
        print('epoch %d: train_loss = %.4f, val_loss = %.4f' % (e+1, loss_train[e], loss_val[e]))
    
    x = range(1, epochs+1)
    plt.plot(x, loss_train, 'b-', label="training loss")
    plt.plot(x, loss_val, 'r--', label="validation loss")
    plt.xlabel("Epochs")
    plt.ylabel("MSE Loss")
    plt.title("Train/Validation Loss")
    plt.legend()
    plt.show()

def prediction(model, x):   # only a batch
    model = model.to(device=device)
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        x = x.to(device=device, dtype=torch.float32)
        score = torch.Tensor.cpu(model(x))
    return score